In [9]:
import pandas as pd
# Specify the path to your Feather file
df = pd.read_feather('/home/bkcs/Desktop/TrAdaBoost/Concatenated_small/MultipleSource/Test_90.feather') 
df

,flow_id,0,1,2,3,4,5,6,7,8,...,247,248,249,250,251,252,253,254,255,Label
0,3418,23,3,3,0,30,25,17,48,29,...,0,0,0,0,0,0,0,0,0,0
1,3418,23,3,3,0,94,21,118,136,26,...,0,0,0,0,0,0,0,0,0,0
2,3418,23,3,3,0,30,159,250,72,95,...,0,0,0,0,0,0,0,0,0,0
3,3418,23,3,3,0,154,85,85,156,61,...,0,0,0,0,0,0,0,0,0,0
4,3418,23,3,3,0,47,122,79,32,128,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160335,229699,170,69,199,73,210,121,122,70,249,...,79,168,225,147,168,111,99,30,243,2
160336,229699,239,232,213,237,234,54,151,236,117,...,187,166,90,22,134,70,168,12,99,2
160337,229699,159,88,250,253,34,147,187,210,85,...,0,0,0,0,0,0,0,0,0,2
160338,229699,252,109,125,110,123,224,147,101,247,...,63,235,146,9,14,203,116,238,43,2


In [10]:
NUM_FEATURE = 256

def data_processing(df, NUM_FEATURES):
   y_train = df['Label']
   flow_id = df['flow_id']

   df = df/255

   X_train = df.drop(['Label', 'flow_id'], axis=1)
   X_train = X_train.to_numpy() / 255
#    NUM_FEATURES = X_train.shape[1]
  
   # nhom 20packet thanh 1 flow
   X_train = X_train.reshape(-1,20, NUM_FEATURES)

   y_train = y_train.to_numpy()

   # lay nhan cuoi cung
   y_train = y_train.reshape(-1,20)[:,-1] 
   return X_train, y_train

X, y = data_processing(df, NUM_FEATURE)

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F
import pandas as pd
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=5, padding=2)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(64, 64, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(64 * 5 * 64, 256)
        self.fc2 = nn.Linear(256, 3)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.pool(x)
        x = self.dropout(x)
        x = x.view(-1, 64 * 5 * 64)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

trans_data = torch.tensor(X_train, dtype=torch.float32)
trans_label = torch.tensor(y_train, dtype=torch.long)
P = torch.rand((len(X_train), 1), dtype=torch.float32)

# Set hyperparameters
NUM_FEATURE = 256
NUM_CLASSES = 3
PACKET_NUM = 20
client_lr = 3e-4
NUM_EPOCHS = 10
BATCH_SIZE = 16

# Create DataLoader
train_dataset = TensorDataset(trans_data.unsqueeze(1), trans_label, P)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Initialize model, loss function, and optimizer
model = Net()
optimizer = optim.Adam(model.parameters(), lr=client_lr)
criterion = nn.CrossEntropyLoss()

# Training loop
for epoch in range(NUM_EPOCHS):
    running_loss = 0.0
    for batch_data, batch_label, batch_P in train_loader:
        optimizer.zero_grad()
        output = model(batch_data)
        loss = criterion(output, batch_label)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch + 1}/{NUM_EPOCHS}], Loss: {running_loss / len(train_loader)}")

# Save the trained model if needed
# torch.save(model.state_dict(), 'your_model.pth')


Epoch [1/10], Loss: 1.0199694648347888
Epoch [2/10], Loss: 1.0173048973083496
Epoch [3/10], Loss: 1.0171601848114755
Epoch [4/10], Loss: 1.0169215755272387
Epoch [5/10], Loss: 1.017147732643118
Epoch [6/10], Loss: 1.018206610643953
Epoch [7/10], Loss: 1.0162014879490668
Epoch [8/10], Loss: 1.0184224276174036
Epoch [9/10], Loss: 1.0160760325089357
Epoch [10/10], Loss: 1.0156983587509973
